In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-eastus-rg')

In [2]:
import git

# get root of git repo
prefix = git.Repo('.', search_parent_directories=True).working_tree_dir

# training script 
script_dir = prefix+'/code/models/fastai/pets-resnet34/'
script_name = 'train.py'

# environment file
environment_file = prefix+'/environments/fastai-example.dockerfile'

# azure ml settings
environment_name = 'fastai-pets-example'
experiment_name = 'fastai-pets-example'
compute_target = 'gpu-cluster'

In [3]:
print(open(script_dir+script_name).read())

import mlflow.fastai 
import fastai.vision as vis

path = untar_data(vis.URLs.PETS)

files = get_image_files(path/'images')

def label_func(f): 
    return f[0].isupper()

# To get our data ready for a model, we need to put it in a DataLoaders object. 
# Here we have a function that labels using the file names, so we will use 
# ImageDataLoaders.from_name_func. There are other factory methods of ImageDataLoaders 
# that could be more suitable for your problem, so make sure to check them all in vision.data.

dls = vis.ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))

# We have passed to this function the directory we're working in, the files we grabbed, our 
# label_func and one last piece as item_tfms: this is a Transform applied on all items of our 
# dataset that will resize each imge to 224 by 224, by using a random crop on the largest 
# dimension to make it a square, then resizing to 224 by 224. If we didn't pass this, we 
# would get an error later 

In [14]:
from azureml.core import ScriptRunConfig, Experiment, Environment

env = Environment(environment_name)
env.docker.enabled = True
env.docker.base_image = None
env.docker.base_dockerfile = environment_file
env.python.user_managed_dependencies = True

src = ScriptRunConfig(source_directory=script_dir, script=script_name)
src.run_config.environment = env
src.run_config.target = compute_target

run = Experiment(ws, experiment_name).submit(src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
fastai-pets-example,fastai-pets-example_1598903614_4e3894a9,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [ ]:
run.wait_for_completion(show_output=True)